In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 9: Disk Fragmenter ---</h2><p>Another push of the button leaves you in the familiar hallways of some friendly <a href="/2021/day/23">amphipods</a>! Good thing you each somehow got your own personal mini submarine. The Historians jet away in search of the Chief, mostly by driving directly into walls.</p>
<p>While The Historians quickly figure out how to pilot these things, you notice an amphipod in the corner struggling with his computer. He's trying to make more contiguous free space by compacting all of the files, but his program isn't working; you offer to help.</p>
<p>He shows you the <em>disk map</em> (your puzzle input) he's already generated. For example:</p>
<pre><code>2333133121414131402</code></pre>
<p>The disk map uses a dense format to represent the layout of <em>files</em> and <em>free space</em> on the disk. The digits alternate between indicating the length of a file and the length of free space.</p>
<p>So, a disk map like <code>12345</code> would represent a one-block file, two blocks of free space, a three-block file, four blocks of free space, and then a five-block file. A disk map like <code>90909</code> would represent three nine-block files in a row (with no free space between them).</p>
<p>Each file on disk also has an <em>ID number</em> based on the order of the files as they appear <em>before</em> they are rearranged, starting with ID <code>0</code>. So, the disk map <code>12345</code> has three files: a one-block file with ID <code>0</code>, a three-block file with ID <code>1</code>, and a five-block file with ID <code>2</code>. Using one character for each block where digits are the file ID and <code>.</code> is free space, the disk map <code>12345</code> represents these individual blocks:</p>
<pre><code>0..111....22222</code></pre>
<p>The first example above, <code>2333133121414131402</code>, represents these individual blocks:</p>
<pre><code>00...111...2...333.44.5555.6666.777.888899</code></pre>
<p>The amphipod would like to <em>move file blocks one at a time</em> from the end of the disk to the leftmost free space block (until there are no gaps remaining between file blocks). For the disk map <code>12345</code>, the process looks like this:</p>
<pre><code>0..111....22222
02.111....2222.
022111....222..
0221112...22...
02211122..2....
022111222......
</code></pre>
<p>The first example requires a few more steps:</p>
<pre><code>00...111...2...333.44.5555.6666.777.888899
009..111...2...333.44.5555.6666.777.88889.
0099.111...2...333.44.5555.6666.777.8888..
00998111...2...333.44.5555.6666.777.888...
009981118..2...333.44.5555.6666.777.88....
0099811188.2...333.44.5555.6666.777.8.....
009981118882...333.44.5555.6666.777.......
0099811188827..333.44.5555.6666.77........
00998111888277.333.44.5555.6666.7.........
009981118882777333.44.5555.6666...........
009981118882777333644.5555.666............
00998111888277733364465555.66.............
0099811188827773336446555566..............
</code></pre>
<p>The final step of this file-compacting process is to update the <em>filesystem checksum</em>. To calculate the checksum, add up the result of multiplying each of these blocks' position with the file ID number it contains. The leftmost block is in position <code>0</code>. If a block contains free space, skip it instead.</p>
<p>Continuing the first example, the first few blocks' position multiplied by its file ID number are <code>0 * 0 = 0</code>, <code>1 * 0 = 0</code>, <code>2 * 9 = 18</code>, <code>3 * 9 = 27</code>, <code>4 * 8 = 32</code>, and so on. In this example, the checksum is the sum of these, <code><em>1928</em></code>.</p>
<p><span title="Bonus points if you make a cool animation of this process.">Compact the amphipod's hard drive</span> using the process he requested. <em>What is the resulting filesystem checksum?</em> <span class="quiet">(Be careful copy/pasting the input for this puzzle; it is a single, very long line.)</span></p>
</article>


In [2]:
tests = [
    {
        "name": "Small Example",
        "s": "12345",
        "expected": 60,
    },
    {
        "name": "Example",
        "s": "2333133121414131402",
        "expected": 1928,
    },
]


class FileSystem(Str):
    def __init__(self, s: str) -> None:
        self.disk = self._decompress(s.strip())

    def _decompress(self, s: str) -> list[str]:
        disk = []
        id = 0
        for i, ch in enumerate(s):
            n = int(ch)
            if i & 1 == 0:
                disk.extend(n * [id])
                id += 1
            else:
                disk.extend(n * ".")

        return disk

    def defragment(self) -> FileSystem:
        left, right = 0, len(self.disk) - 1

        while True:
            while left < right and not isinstance(self.disk[right], int):
                right -= 1

            while left < right and self.disk[left] != ".":
                left += 1

            if left >= right:
                break

            self.disk[left], self.disk[right] = self.disk[right], self.disk[left]

        return self

    def checksum(self) -> int:
        return sum(i * v for i, v in enumerate(self.disk) if v != ".")

    def __str__(self) -> str:
        return f"{type(self).__name__}({
            ', '.join(f'{n}={''.join(map(str,v))}' for n, v in vars(self).items())
        })"


@test(tests=tests[:])
def partI_test(s: str) -> int:
    return FileSystem(s).defragment().checksum()


Test Small Example passed, for partI_test.
Test Example passed, for partI_test.
Success


In [3]:
with open("../input/day9.txt") as f:
    puzzle = f.read()

print(f"Part I: {FileSystem(puzzle).defragment().checksum()}")

Part I: 6349606724455


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>6349606724455</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Upon completion, two things immediately become clear. First, the disk definitely has a lot more contiguous free space, just like the amphipod hoped. Second, the computer is running much more slowly! Maybe introducing all of that <a href="https://en.wikipedia.org/wiki/File_system_fragmentation" target="_blank">file system fragmentation</a> was a bad idea?</p>
<p>The eager amphipod already has a new plan: rather than move individual blocks, he'd like to try compacting the files on his disk by moving <em>whole files</em> instead.</p>
<p>This time, attempt to move whole files to the leftmost span of free space blocks that could fit the file. Attempt to move each file exactly once in order of <em>decreasing file ID number</em> starting with the file with the highest file ID number. If there is no span of free space to the left of a file that is large enough to fit the file, the file does not move.</p>
<p>The first example from above now proceeds differently:</p>
<pre><code>00...111...2...333.44.5555.6666.777.888899
0099.111...2...333.44.5555.6666.777.8888..
0099.1117772...333.44.5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
</code></pre>
<p>The process of updating the filesystem checksum is the same; now, this example's checksum would be <code><em>2858</em></code>.</p>
<p>Start over, now compacting the amphipod's hard drive using this new method instead. <em>What is the resulting filesystem checksum?</em></p>
</article>


In [4]:
from collections import deque

from sortedcollections import SortedDict, SortedList


tests = [
    {
        "name": "Small Example",
        "s": "12345",
        "expected": 132,
    },
    {
        "name": "Example",
        "s": "2333133121414131402",
        "expected": 2858,
    },
]


class FileSystemII(FileSystem):
    def __init__(self, s: str) -> None:
        super().__init__(s)
        self.compact_start = s.strip()

    def freespace(
        self, s: str
    ) -> tuple[dict[int, SortedList[int]], list[tuple[int, int]]]:
        free = defaultdict(SortedList[int])
        files = []

        idx = 0
        for i, c in enumerate(map(int, s.strip())):
            if i & 1 == 1:
                free[c].add(idx)
            else:
                files.append((idx, c))
            idx += c

        return dict(free), files

    def defragment(self) -> FileSystem:
        free, files = self.freespace(self.compact_start)
        lengths = SortedList(free.keys())
        print(repr(self))

        while files:
            ifile, file_length = files.pop()
            file_id = self.disk[ifile]

            ib = lengths.bisect_left(file_length)

            if 0 < ib < len(lengths) and ifile >= free[lengths[ib]][0]:
                self.disk[ifile : ifile + file_length] = ["."] * file_length
                free_length = lengths[ib]
                iempty = free[free_length].pop(0)

                self.disk[iempty : iempty + file_length] = [file_id] * file_length
                if free_length > file_length:
                    new_free_idx = iempty + file_length
                    new_lenght = free_length - file_length

                    if new_lenght not in free:
                        free[new_lenght] = SortedList()
                        lengths.add(new_lenght)

                    free[new_lenght].add(new_free_idx)

                if not free[free_length]:
                    free.pop(free_length)
                    lengths.pop(ib)
        print(repr(self))
        return self


@test(tests=tests[:])
def partII_test(s: str) -> int:
    return FileSystemII(s).defragment().checksum()


FileSystemII(disk=[0, '.', '.', 1, 1, 1, '.', '.', '.', '.', 2, 2, 2, 2, 2], compact_start=12345)
FileSystemII(disk=[0, '.', '.', 1, 1, 1, '.', '.', '.', '.', 2, 2, 2, 2, 2], compact_start=12345)
Test Small Example passed, for partII_test.
FileSystemII(disk=[0, 0, '.', '.', '.', 1, 1, 1, '.', '.', '.', 2, '.', '.', '.', 3, 3, 3, '.', 4, 4, '.', 5, 5, 5, 5, '.', 6, 6, 6, 6, '.', 7, 7, 7, '.', 8, 8, 8, 8, 9, 9], compact_start=2333133121414131402)
FileSystemII(disk=[0, 0, 9, 9, 2, 1, 1, 1, 7, 7, 7, '.', 4, 4, '.', 3, 3, 3, '.', '.', '.', '.', 5, 5, 5, 5, '.', 6, 6, 6, 6, '.', '.', '.', '.', '.', 8, 8, 8, 8, '.', '.'], compact_start=2333133121414131402)
Test Example passed, for partII_test.
Success


<link href="style.css" rel="stylesheet"></link>


In [5]:
print(f"Part II: {FileSystemII(puzzle).defragment().checksum()}")

FileSystemII(disk=[0, 0, 0, '.', '.', '.', '.', '.', '.', '.', '.', 1, 1, 1, 1, 1, 1, '.', '.', 2, 2, 2, 2, 2, 2, '.', '.', '.', '.', '.', '.', 3, '.', '.', '.', '.', '.', '.', 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, '.', '.', '.', '.', '.', '.', '.', 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, '.', '.', '.', '.', '.', 8, 8, 8, 8, 8, 8, 8, '.', 9, 9, 9, 9, 9, 9, 9, '.', '.', '.', '.', '.', '.', '.', '.', 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, '.', '.', '.', '.', '.', '.', '.', '.', 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, '.', '.', '.', '.', '.', '.', 14, 14, 14, 14, 14, 14, 14, 14, 14, '.', '.', '.', '.', 15, 15, 15, 15, '.', '.', '.', 16, 16, 16, 16, 16, 16, 16, 16, '.', '.', '.', '.', '.', '.', '.', '.', 17, 17, '.', '.', '.', 18, 18, 18, 18, 18, 18, 18, '.', 19, 19, '.', '.', 20, 20, 20, 21, 21, 21, 21, '.', '.', '.', 22, 22, 22, 22, 22, 22, 22, 22, '.', '.', '.', '.', '.', '.', '.', 23, 23, '.', '.', '.', '.', '.', '.', 24, 24, 24, 24, 24, 24

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1221</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
